# **MÓDULO 32 - Exercício**
# Random Forest


Nesta tarefa, vocês vão trabalhar com uma base de dados de avaliações de vinhos, onde o objetivo é prever a pontuação dos vinhos usando o algoritmo de Random Forest para classificação multiclasse.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("winequality-red.csv", delimiter=',')

df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5


**Vamos conhecer nossa base:**

Características dos Vinhos (Features)

Fixed Acidity: Acidez fixa do vinho.

Volatile Acidity: Acidez volátil do vinho.

Citric Acid: Quantidade de ácido cítrico no vinho.

Residual Sugar: Açúcar residual presente no vinho.

Chlorides: Nível de cloretos no vinho.

Free Sulfur Dioxide: Dióxido de enxofre livre no vinho.

Total Sulfur Dioxide: Quantidade total de dióxido de enxofre no vinho.

Density: Densidade do vinho.

pH: Nível de pH do vinho.

Sulphates: Quantidade de sulfatos no vinho.

Alcohol: Teor alcoólico do vinho.



**Variável de Saída (Target):**

Quality: Pontuação do vinho baseada em dados sensoriais, variando de 0 a 10.


Esta abordagem permitirá que vocês explorem como diferentes características químicas influenciam a qualidade dos vinhos e como o Random Forest pode ser usado para fazer previsões precisas com base nesses dados.

# 1 - Realize a primeira etapa de pré processamento dos dados.

A) Verifique os tipos de dados.


B) Verifique os dados faltantes, se houver dados faltantes faça a substituição ou remoção justificando sua escolha.

In [ ]:
df.dtypes

,0
fixed acidity,float64
volatile acidity,float64
citric acid,float64
residual sugar,float64
chlorides,float64
free sulfur dioxide,float64
total sulfur dioxide,float64
density,float64
pH,float64
sulphates,float64


In [ ]:
df.isnull().sum()

,0
fixed acidity,0
volatile acidity,0
citric acid,0
residual sugar,0
chlorides,0
free sulfur dioxide,0
total sulfur dioxide,0
density,0
pH,0
sulphates,0


# 2 - Realize a segunda e terceita etapa de pré processamento dos dados.

A) Utilize a função describe para identificarmos outliers e verificarmos a distribuição dos dados.

B) Verifique o balanceamento da váriavel Target.

C)  Plote o gráfico ou a tabela e indique as variáveis que te parecem mais "fortes" na correlação para nosso modelo.

D) Crie um novo dataframe apenas com as váriaveis que parecem ter maior correlação com a target. (Negativa ou positiva)


In [ ]:
#@title 2.A - Análise estatística
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


A análise estatística via describe() mostrou valores extremos em algumas variáveis, mas esses outliers são naturais do processo químico dos vinhos e não representam erro. Como o modelo Random Forest é robusto a outliers, optou-se por manter todos os valores originais.

In [ ]:
#@title 2.B - Verificação do balanceamento da variável Target

df['quality'].value_counts().sort_index()

,count
quality,
3,10
4,53
5,681
6,638
7,199
8,18


A variável quality é desbalanceada, com maior concentração nas classes 5, 6 e 7.
Apesar disso, não é obrigatório aplicar técnicas de balanceamento, pois o Random Forest lida bem com esse tipo de distribuição.

In [ ]:
#@title 2.C — Correlação entre variáveis e identificação das mais fortes

df.corr()['quality'].sort_values(ascending=False)

,quality
quality,1.000000
alcohol,0.476166
sulphates,0.251397
citric acid,0.226373
fixed acidity,0.124052
residual sugar,0.013732
free sulfur dioxide,-0.050656
pH,-0.057731
chlorides,-0.128907
density,-0.174919


Variáveis com maior correlação com a qualidade (positiva ou negativa):

alcohol (+): maior influência positiva

volatile acidity (–): maior influência negativa

sulphates (+): relação moderada

citric acid (+): leve relação positiva

density (–): relação negativa moderada

total sulfur dioxide (–): leve influência negativa

As demais variáveis têm pouca influência direta no target.

In [ ]:
#@title 2.D — Criação de dataframe com as variáveis mais correlacionadas

df_correlacoes = df[[
    'alcohol',
    'volatile acidity',
    'sulphates',
    'citric acid',
    'density',
    'total sulfur dioxide',
    'quality'
]]

df_correlacoes.head()

,alcohol,volatile acidity,sulphates,citric acid,density,total sulfur dioxide,quality
0,9.4,0.70,0.56,0.00,0.9978,34.0,5
1,9.8,0.88,0.68,0.00,0.9968,67.0,5
2,9.8,0.76,0.65,0.04,0.9970,54.0,5
3,9.8,0.28,0.58,0.56,0.9980,60.0,6
4,9.4,0.70,0.56,0.00,0.9978,34.0,5


# 3 - Preparação Final dos Dados

A) Separe a base em X(Features) e Y(Target)

B) Separe a base em treino e teste.


In [ ]:
#@title 3.A — Separação em X e Y
X = df_correlacoes.drop('quality', axis=1)
Y = df_correlacoes['quality']

In [ ]:
#@title 3.B — Separação em treino e teste

X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.25,
    random_state=42,
    stratify=Y
)

# 4 - Modelagem

A) Inicie e treine o modelo de Random Forest

B) Aplique a base de teste o modelo.


In [ ]:
#@title 4.A — Iniciar e treinar o modelo Random Forest

modelo = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)

modelo.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
#@title 4.B — Aplicar o modelo na base de teste
previsoes = modelo.predict(X_test)

# 5 - Avaliação

A) Avalie as principais métricas da Claissificação e traga insights acerca do resultado, interprete os valores achados.

B) Você nota que o modelo teve dificuldade para prever alguma classe? Se sim, acredita que tenha relação com o balanceamento dos dados? Explique.


In [ ]:
acc = accuracy_score(Y_test, previsoes)
relatorio = classification_report(
    Y_test,
    previsoes,
    zero_division=0
)
matriz = confusion_matrix(Y_test, previsoes)

acc, relatorio, matriz


(0.66,
 '              precision    recall  f1-score   support\n\n           3       0.00      0.00      0.00         2\n           4       0.00      0.00      0.00        13\n           5       0.71      0.76      0.74       170\n           6       0.61      0.71      0.66       160\n           7       0.67      0.40      0.50        50\n           8       0.50      0.20      0.29         5\n\n    accuracy                           0.66       400\n   macro avg       0.42      0.35      0.36       400\nweighted avg       0.64      0.66      0.64       400\n',
 array([[  0,   0,   1,   1,   0,   0],
        [  0,   0,  11,   2,   0,   0],
        [  0,   0, 129,  41,   0,   0],
        [  0,   1,  38, 114,   7,   0],
        [  0,   0,   2,  27,  20,   1],
        [  0,   0,   0,   1,   3,   1]]))

O modelo teve acurácia de 0.66, com bom desempenho nas classes mais comuns (5 e 6), porque são as que possuem mais exemplos no conjunto de dados. As classes 7 tiveram desempenho intermediário, já que muitos vinhos 7 foram previstos como 6, o que é esperado pela semelhança entre essas notas.

As classes 3, 4 e 8 não foram previstas corretamente porque têm pouquíssimas amostras. O modelo não consegue aprender padrões com tão poucos exemplos, e isso está diretamente relacionado ao desbalanceamento da variável qualidade. Por isso, precisão e recall ficaram zerados nessas classes.

# 6 - Melhorando os Hyperparametros

A) Defina o Grid de parametros que você quer testar

B) Inicie e Treine um novo modelo utilizando o random search.

C) Avalie os resultados do modelo.

D) Você identificou melhorias no modelo após aplicar o random search? Justifique.


ps. Essa parte da atividade demorará um pouco para rodar!

In [ ]:
#@title 6.A — Definição do Grid de Hiperparâmetros

parametros = {
    'n_estimators': [200, 300, 400, 600, 800],
    'max_depth': [10, 15, 20, 25, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'max_features': ['sqrt', 'log2', None]
}

In [ ]:
#@title 6.B — Random Search com mais iterações (Treino do Modelo)

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

modelo_base = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(
    estimator=modelo_base,
    param_distributions=parametros,
    n_iter=50,
    cv=3,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, Y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 15, 20, 25, None],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10, 15],
                                        'n_estimators': [200, 300, 400, 600,
                                                         800]},
                   random_state=42)

In [ ]:
#@title 6.C — Avaliação do Modelo Otimizado

melhor_modelo = random_search.best_estimator_
previsoes_random = melhor_modelo.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

acc_random = accuracy_score(Y_test, previsoes_random)

relatorio_random = classification_report(
    Y_test,
    previsoes_random,
    zero_division=0
)

matriz_random = confusion_matrix(Y_test, previsoes_random)

acc_random, relatorio_random, matriz_random

(0.6625,
 '              precision    recall  f1-score   support\n\n           3       0.00      0.00      0.00         2\n           4       0.00      0.00      0.00        13\n           5       0.71      0.76      0.74       170\n           6       0.62      0.71      0.66       160\n           7       0.67      0.40      0.50        50\n           8       0.50      0.20      0.29         5\n\n    accuracy                           0.66       400\n   macro avg       0.42      0.35      0.36       400\nweighted avg       0.64      0.66      0.64       400\n',
 array([[  0,   0,   1,   1,   0,   0],
        [  0,   0,  11,   2,   0,   0],
        [  0,   0, 130,  40,   0,   0],
        [  0,   1,  38, 114,   7,   0],
        [  0,   0,   2,  27,  20,   1],
        [  0,   0,   0,   1,   3,   1]]))

Após aplicar o Random Search com um grid ampliado e maior número de iterações, o modelo apresentou uma melhora muito pequena, elevando a acurácia apenas para 0.6625, praticamente igual ao modelo inicial. As métricas mostram que o desempenho nas classes mais frequentes (5 e 6) permaneceu parecido, e a classe 7 teve leve oscilação, sem ganho significativo.

As classes raras (3, 4 e 8) continuaram com precisão e recall iguais a zero, ou próximos disso, demonstrando que mesmo com hiperparâmetros otimizados o modelo não consegue aprender padrões suficientes nessas categorias. Isso ocorre devido ao forte desbalanceamento da base e ao número extremamente baixo de exemplos nessas classes, o que limita completamente a capacidade do modelo de melhorar nesses casos.

Portanto, o Random Search trouxe apenas ajustes finos no comportamento geral, mas não resultou em uma melhoria substancial no desempenho, pois o principal problema não está nos hiperparâmetros e sim na distribuição desigual das classes na variável alvo.

# 7 - Chegando a perfeição

Baseado em tudo que você já aprendeu até agora, quais outras técnicas você acredita que poderiam ser aplicadas ao modelo para melhorar ainda mais suas previsões?

Durante o desenvolvimento do modelo, testamos várias técnicas para melhorar a previsão da qualidade dos vinhos. Primeiro analisamos as correlações por classe, o que ajudou a entender quais variáveis químicas influenciam mais cada faixa de qualidade. A partir disso criamos novas variáveis derivadas que combinam acidez, densidade, álcool e pH, deixando o modelo mais próximo da lógica usada pelos avaliadores humanos.

Mesmo após testar diferentes hiperparâmetros e pesos de classe, o ganho foi pequeno. A melhora significativa veio com o agrupamento das notas em faixas de qualidade (ruim, média e boa), que reduz o impacto do desbalanceamento extremo da base. Com essa abordagem, a acurácia subiu de cerca de 66% para ~87%, mostrando que transformar o problema em uma classificação por grupos torna o modelo muito mais estável.

No fim, as técnicas que realmente ajudaram foram:

criar variáveis derivadas baseadas na correlação química,

usar todas as features combinadas,

e aplicar o agrupamento da variável alvo para reduzir ruído e melhorar a generalização.

In [37]:
classes = sorted(df["quality"].unique())
correlacoes_por_classe = {}

for c in classes:
    df_classe = df[df["quality"] == c].drop("quality", axis=1)
    corr = df_classe.corr()
    correlacoes_por_classe[c] = corr

    print(f"\nCORRELAÇÃO DA CLASSE {c}\n")

    corr_filtrada = corr[(corr >= 0.4) | (corr <= -0.4)]
    print(corr_filtrada)


CORRELAÇÃO DA CLASSE 3

                      fixed acidity  volatile acidity  citric acid  \
fixed acidity              1.000000         -0.578216     0.962292   
volatile acidity          -0.578216          1.000000    -0.692684   
citric acid                0.962292         -0.692684     1.000000   
residual sugar                  NaN               NaN          NaN   
chlorides                       NaN               NaN          NaN   
free sulfur dioxide             NaN         -0.471159          NaN   
total sulfur dioxide       0.474094         -0.539056     0.528138   
density                    0.805218         -0.593561     0.779402   
pH                        -0.586613          0.578023    -0.681522   
sulphates                  0.491068         -0.792241     0.535104   
alcohol                   -0.587774          0.717392    -0.755958   

                      residual sugar  chlorides  free sulfur dioxide  \
fixed acidity                    NaN        NaN               

In [32]:
df_fe = df.copy()

df_fe["acidez_total"] = df_fe["fixed acidity"] + df_fe["volatile acidity"] + df_fe["citric acid"]

df_fe["densidade_acidez"] = df_fe["density"] / df_fe["fixed acidity"]

df_fe["ratio_volatil"] = df_fe["volatile acidity"] / df_fe["fixed acidity"]

df_fe["pH_inverso"] = 1 / df_fe["pH"]

df_fe["qualidade_quimica"] = (
    df_fe["fixed acidity"] +
    df_fe["citric acid"] +
    df_fe["chlorides"] +
    df_fe["density"]
)

df_fe.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,acidez_total,densidade_acidez,ratio_volatil,pH_inverso,qualidade_quimica
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,8.10,0.134838,0.094595,0.284900,8.4738
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,8.68,0.127795,0.112821,0.312500,8.8948
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,8.60,0.127821,0.097436,0.306748,8.9290
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,12.04,0.089107,0.025000,0.316456,12.8330
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,8.10,0.134838,0.094595,0.284900,8.4738


In [33]:
df_modelo = df_fe.copy()

X = df_modelo.drop("quality", axis=1)
y = df_modelo["quality"]

X.head(), y.head()

(   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
 0            7.4              0.70         0.00             1.9      0.076   
 1            7.8              0.88         0.00             2.6      0.098   
 2            7.8              0.76         0.04             2.3      0.092   
 3           11.2              0.28         0.56             1.9      0.075   
 4            7.4              0.70         0.00             1.9      0.076   
 
    free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
 0                 11.0                  34.0   0.9978  3.51       0.56   
 1                 25.0                  67.0   0.9968  3.20       0.68   
 2                 15.0                  54.0   0.9970  3.26       0.65   
 3                 17.0                  60.0   0.9980  3.16       0.58   
 4                 11.0                  34.0   0.9978  3.51       0.56   
 
    alcohol  acidez_total  densidade_acidez  ratio_volatil  pH_inverso  

In [34]:
from sklearn.model_selection import train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

X_treino.shape, X_teste.shape


((1199, 16), (400, 16))

In [40]:
#@title Agrupamento das Classes

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore")

# Criando agrupamento (Ruim = 0, Médio = 1, Bom = 2)
df_grouped = df.copy()

df_grouped['quality_group'] = df_grouped['quality'].apply(
    lambda x: 0 if x <= 4 else (1 if x <= 6 else 2)
)

print("Balanceamento da nova target (quality_group):")
print(df_grouped['quality_group'].value_counts())

# Separando X e y com todas variáveis + derivadas
X = df_grouped.drop(['quality', 'quality_group'], axis=1)
y = df_grouped['quality_group']

# Treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Modelo otimizado
modelo = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    min_samples_split=4,
    min_samples_leaf=2,
    bootstrap=True,
    class_weight=None,
    random_state=42
)

modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

acc = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
matriz = confusion_matrix(y_test, y_pred)

acc, report, matriz


Balanceamento da nova target (quality_group):
quality_group
1    1319
2     217
0      63
Name: count, dtype: int64


(0.875,
 '              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00        16\n           1       0.89      0.96      0.93       330\n           2       0.73      0.59      0.65        54\n\n    accuracy                           0.88       400\n   macro avg       0.54      0.52      0.53       400\nweighted avg       0.84      0.88      0.85       400\n',
 array([[  0,  16,   0],
        [  0, 318,  12],
        [  0,  22,  32]]))